# Import packages and database

In [2]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, Derive
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
from collections import namedtuple

data_file ='https://raw.githubusercontent.com/GustavePellier/MMOB/main/lpmc19.dat'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC

database = db.Database('LPMC', LPMC)
all_results = {}

C:\Users\gusta\anaconda3\envs\MMB\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Model 0

We calculate the total public transport duration and the total driving cost

In [3]:
LPMC["dur_pt"]= LPMC["dur_pt_access"] + LPMC["dur_pt_rail"] + LPMC["dur_pt_bus"] + LPMC["dur_pt_int"] 
LPMC["cost_drive"] = LPMC["cost_driving_ccharge"] + LPMC["cost_driving_fuel"]

Some variables are created with the columns that seem to be useful

In [4]:
travel_mode=Variable('travel_mode')

dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')

pt_interchanges=Variable('pt_interchanges')

cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_drive=Variable('cost_drive')
cost_pt=Variable('cost_transit')

time_walk=Variable('dur_walking')
time_cycle=Variable('dur_cycling')
time_pt=Variable('dur_pt')
time_drive=Variable('dur_driving')

female=Variable('female')
age=Variable('age')

There are 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [5]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

Utility functions creation

In [6]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

availability of each mode, all available here

In [7]:
av = {1: 1, 2: 1, 3: 1, 4:1}

The estimation results (parameter values, t-tests or p-values, null and final log likelihoods)

In [8]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.878619,0.107672,-36.022418,0.0
asc_drive,-1.295191,0.080505,-16.088354,0.0
asc_pt,-0.503354,0.054123,-9.300119,0.0
beta_cost,-0.193629,0.013958,-13.871889,0.0
beta_time,-5.495527,0.208596,-26.345342,0.0


In [9]:
res.compile_estimation_results(all_results)

(                                         Model0
 Number of estimated parameters                5
 Sample size                                5000
 Final log likelihood               -4587.818071
 Akaike Information Criterion        9185.636142
 Bayesian Information Criterion      9218.222108
 asc_cycle (t-test)                 -3.88  (-36)
 asc_drive (t-test)                -1.3  (-16.1)
 asc_pt (t-test)                  -0.503  (-9.3)
 beta_cost (t-test)              -0.194  (-13.9)
 beta_time (t-test)                -5.5  (-26.3),
 {'Model0': 'Model0'})

In [10]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1

We choose to do a specification for the time attribute, the cost stays generic in this model. Therefore we obtain a beta_time parameter for each transportation mode. 

In [11]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

The utility functions are rewritten with the new parameters :

In [12]:
v_walk_model1= beta_time_walk * time_walk  
v_cycle_model1= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1= asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1= asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

The results are computed (parameter values, t-tests or p-values, null and final log likelihoods)

In [13]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model_1'
all_results['Model1'] = biogeme_model1.estimate()
results_specific_time = biogeme_model1.estimate()
results_specific_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.847224,0.202570,-23.928654,0.0
asc_drive,-1.969873,0.137929,-14.281808,0.0
asc_pt,-2.337196,0.139506,-16.753342,0.0
beta_cost,-0.182008,0.016087,-11.314047,0.0
beta_time_cycle,-5.312452,0.462002,-11.498762,0.0
beta_time_drive,-6.474307,0.378989,-17.083106,0.0
beta_time_pt,-3.538619,0.250656,-14.117426,0.0
beta_time_walk,-8.430765,0.418990,-20.121621,0.0


In [14]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1
 Number of estimated parameters                5                8
 Sample size                                5000             5000
 Final log likelihood               -4587.818071     -4275.496542
 Akaike Information Criterion        9185.636142      8566.993084
 Bayesian Information Criterion      9218.222108       8619.13063
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)
 beta_time (t-test)                -5.5  (-26.3)                 
 beta_time_cycle (t-test)                          -5.31  (-11.5)
 beta_time_drive (t-test)                          -6.47  (-17.1)
 beta_time_pt (t-test)                             -3.54  (-14.1)
 beta_time_walk (t-test)                           -8.43  (-20.1),
 {'Model0

In [15]:
biogeme_model1.calculateNullLoglikelihood(av)

-6931.471805599917

We compare the 2 models with the likelihood ratio test because model 0 is a reduction of model 1.  

In [20]:
old  = results_generic
new  = results_specific_time
new.likelihood_ratio_test(old, 0.01)

LRTuple(message='H0 can be rejected at level 1.0%', statistic=624.6430583499878, threshold=11.344866730144373)

# Model 2